In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_log_error

Tricks on optimization when you have large datasets
1. FOR THE LOVE OF GOD, PLEASE DON'T USE CROSS-VALIDATION. The whole point of just using a validation set (vs cross-validation) is that it requires less computational time. Which is good when you have huge datasets as we do here.
2. You should only do this if you really need to, but you can simply optimize on a sample of the training set rather than on the full dataset. Once you found the best parameters on the sample you simply need to retrain it on the full training set.

Of course, ideally, you should always use cross-validation and you should also always use the full dataset rather than on sample. But it is not always possible.

We will be using both of these tricks here.

Note that the results here was done on Colab, and colab doesn't have a very good CPU. It was also all done within 1-2 hours, which is pretty quick. Moreover, if you can get XGBoost working with the GPU it should be even quicker. So in your assignments it should be expected that you run this code for even longer.

In [2]:
perth = pd.read_csv('perth_clean.csv')
perth = pd.get_dummies(perth, columns=['suburb'])

train_indices, test_indices = train_test_split(perth.index, test_size=0.2, random_state=0)

perth_train = perth.loc[train_indices].copy()
perth_test = perth.loc[test_indices].copy()

In [3]:
print("Training set length", len(perth_train))
print("Tresting set length", len(perth_test))

Training set length 26924
Tresting set length 6732


In [4]:
np.random.seed(5)

In [9]:
indices = list(perth_train.index.copy())
# np.random.seed(5)
np.random.shuffle(indices)

sampled_indices = indices[:int(len(indices) * 0.2)]
sampled_perth = perth_train.loc[sampled_indices, :].copy()

print("Sampled set length", len(sampled_indices))

Sampled set length 5384


In [10]:
sampled_x_train = sampled_perth.drop('log10_price', axis=1)
sampled_y_train = sampled_perth['log10_price']

In [11]:
n = len(sampled_perth)
sampled_train_indices, sampled_valid_indices = train_test_split(np.arange(n), test_size=0.2, random_state=0)

In [17]:
print("Training length of sampled data", len(sampled_train_indices))
print("Validiation length of sampled data", len(sampled_valid_indices))

Training length of sampled data 4307
Validiation length of sampled data 1077


Just to give a brief recap of what we are doing here. 

1. Because of how big the dataset is, rather than using the entire training set (26924 length) we instead want to use a subset of that. I call this subset the sampled perth set (5384 length)
2. On this sampled perth set you can then choose to use CV or just the validation approach. Because of how long the models take to train, we will be using the validation approach. 

    So the sampled perth set (5384 length) is then split into the sampled training set (4307 length) and sampled validation set (1077 length). We will then choose the best parameters based on this validation set (1077 length) 
3. Finally once we have choose the best hyper-parameter, we will need to retrain it on the full training set (26924 length). 

# Random Forests

In [ ]:
parameter_grid = {'min_samples_leaf': np.arange(1, 100, 5),
                  'max_features': np.arange(1, len(perth_train.columns) - 1, 5)}

base_parameters = {key: np.random.choice(val) for key, val in parameter_grid.items()}
base_parameters

{'max_features': 6, 'min_samples_leaf': 36}

In [ ]:
print(base_parameters)
for parameter, grid in parameter_grid.items():
    base_model = RandomForestRegressor(criterion='squared_error', random_state=0, **base_parameters)
    
    parameter_sub_grid = {parameter: grid}
    search = GridSearchCV(base_model, parameter_sub_grid, 
                          cv=[(sampled_train_indices, sampled_valid_indices)], 
                          scoring="neg_mean_squared_error", verbose=3, refit=False)
    
    search.fit(sampled_x_train, sampled_y_train)
    
    base_parameters.update(search.best_params_)
    print(base_parameters)

{'min_samples_leaf': 36, 'max_features': 6}
Fitting 1 folds for each of 20 candidates, totalling 20 fits
[CV 1/1] END ...............min_samples_leaf=1;, score=-0.011 total time=   1.3s
[CV 1/1] END ...............min_samples_leaf=6;, score=-0.028 total time=   0.3s
[CV 1/1] END ..............min_samples_leaf=11;, score=-0.040 total time=   0.2s
[CV 1/1] END ..............min_samples_leaf=16;, score=-0.045 total time=   0.2s
[CV 1/1] END ..............min_samples_leaf=21;, score=-0.045 total time=   0.2s
[CV 1/1] END ..............min_samples_leaf=26;, score=-0.045 total time=   0.2s
[CV 1/1] END ..............min_samples_leaf=31;, score=-0.045 total time=   0.2s
[CV 1/1] END ..............min_samples_leaf=36;, score=-0.045 total time=   0.2s
[CV 1/1] END ..............min_samples_leaf=41;, score=-0.045 total time=   0.2s
[CV 1/1] END ..............min_samples_leaf=46;, score=-0.045 total time=   0.2s
[CV 1/1] END ..............min_samples_leaf=51;, score=-0.045 total time=   0.2s
[CV 

Again but smarter. So we can simply run this again with the same parameter grid

    parameter_grid = {'min_samples_leaf': np.arange(1, 100, 5),
                      'max_features': np.arange(1, len(perth_train.columns) - 1, 5)}

But when we look at the output a bit more carefully, we can see that for `min_samples_leaf` the error increases drastically after 6. So there is no reason why we need to look any thing more than 6. As such we can focus the search on this interval

    parameter_grid = {'min_samples_leaf': np.arange(1, 6),
                      'max_features': np.arange(1, len(perth_train.columns) - 1, 5)}


For `max_features` we have to be a bit careful. Recall we are doing validation, not cross-validation, as such the validation error tends to be noisy. We can see that for the most part the scores are all $\pm 0.01$ from the best score $0.09$. So we will just leave it as is.

In [ ]:
print(base_parameters)

{'min_samples_leaf': 1, 'max_features': 106}


In [ ]:
parameter_grid = {'min_samples_leaf': np.arange(1, 6),
                  'max_features': np.arange(1, len(perth_train.columns) - 1, 5)}

In [ ]:
print(base_parameters)
for parameter, grid in parameter_grid.items():
    base_model = RandomForestRegressor(criterion='squared_error', random_state=0, **base_parameters)
    
    parameter_sub_grid = {parameter: grid}
    search = GridSearchCV(base_model, parameter_sub_grid, 
                          cv=[(sampled_train_indices, sampled_valid_indices)], 
                          scoring="neg_mean_squared_error", verbose=3, refit=False)
    
    search.fit(sampled_x_train, sampled_y_train)
    
    base_parameters.update(search.best_params_)
    print(base_parameters)

{'min_samples_leaf': 1, 'max_features': 106}
Fitting 1 folds for each of 5 candidates, totalling 5 fits
[CV 1/1] END ...............min_samples_leaf=1;, score=-0.009 total time=   2.7s
[CV 1/1] END ...............min_samples_leaf=2;, score=-0.009 total time=   2.2s
[CV 1/1] END ...............min_samples_leaf=3;, score=-0.010 total time=   1.9s
[CV 1/1] END ...............min_samples_leaf=4;, score=-0.010 total time=   1.8s
[CV 1/1] END ...............min_samples_leaf=5;, score=-0.010 total time=   1.7s
{'min_samples_leaf': 1, 'max_features': 106}
Fitting 1 folds for each of 67 candidates, totalling 67 fits
[CV 1/1] END ...................max_features=1;, score=-0.012 total time=   1.3s
[CV 1/1] END ...................max_features=6;, score=-0.011 total time=   1.2s
[CV 1/1] END ..................max_features=11;, score=-0.011 total time=   1.2s
[CV 1/1] END ..................max_features=16;, score=-0.010 total time=   1.3s
[CV 1/1] END ..................max_features=21;, score=-0.010

### Importantly, once you found the best parameters you need to retrain the final model on the FULL dataset.

As an aside, look how long it takes to train on the full dataset. Now imagine doing this over and over again to find the optimal parameters.

In [ ]:
base_parameters

{'max_features': 106, 'min_samples_leaf': 1}

In [20]:
%%time

final_model = RandomForestRegressor(criterion='squared_error', random_state=0, **base_parameters)
final_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = final_model.predict(perth_test.drop('log10_price', axis=1))

print(np.mean((y - y_pred) ** 2))

0.00781962457241284
CPU times: user 26.2 s, sys: 55 ms, total: 26.2 s
Wall time: 26.2 s


Just to prove the point. We are sampling the dataset when we do the optimization. So it is very likely that the final parameters are not the best when considering the full dataset. But let us see.

In [22]:
# Change min_samples_leaf

new_parameters = {'min_samples_leaf': 6, 'max_features': 106}

new_model = RandomForestRegressor(criterion='squared_error', random_state=0, **new_parameters)
new_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = new_model.predict(perth_test.drop('log10_price', axis=1))

print(np.mean((y - y_pred) ** 2))

0.008252400627787577


In [23]:
# Change max_features

new_parameters = {'min_samples_leaf': 1, 'max_features': 241}

new_model = RandomForestRegressor(criterion='squared_error', random_state=0, **new_parameters)
new_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = new_model.predict(perth_test.drop('log10_price', axis=1))

print(np.mean((y - y_pred) ** 2))

0.007924114544086851


# XGBoost

Note that XGBoost can use a GPU and Colab does give you a free GPU. But I can't figure out how to get that working. So I'll let that for you too look at, but it should make things MUCH MUCH quicker.

In [ ]:
parameter_grid = {'n_estimators': np.arange(50, 100 + 1, 10),
                  'learning_rate': np.linspace(0.01, 0.10, 10),
                  'max_depth': np.arange(1, 100 + 1, 10),
                  'subsample': np.linspace(0.1, 1.0, 10)}

# np.random.seed(5)
base_parameters = {key: np.random.choice(val) for key, val in parameter_grid.items()}
base_parameters

{'learning_rate': 0.06000000000000001,
 'max_depth': 81,
 'n_estimators': 60,
 'subsample': 0.5}

In [ ]:
print(base_parameters)
for parameter, grid in parameter_grid.items():
    base_model = XGBRegressor(objective='reg:squarederror', random_state=0, **base_parameters)
    
    parameter_sub_grid = {parameter: grid}
    search = GridSearchCV(base_model, parameter_sub_grid, 
                          cv=[(sampled_train_indices, sampled_valid_indices)], 
                          scoring="neg_mean_squared_error", verbose=3, refit=False)
    
    search.fit(sampled_x_train, sampled_y_train)
    
    base_parameters.update(search.best_params_)
    print(base_parameters)

{'n_estimators': 60, 'learning_rate': 0.06000000000000001, 'max_depth': 81, 'subsample': 0.5}
Fitting 1 folds for each of 6 candidates, totalling 6 fits
[CV 1/1] END ..................n_estimators=50;, score=-0.070 total time=   3.8s
[CV 1/1] END ..................n_estimators=60;, score=-0.028 total time=   5.9s
[CV 1/1] END ..................n_estimators=70;, score=-0.015 total time=   8.2s
[CV 1/1] END ..................n_estimators=80;, score=-0.011 total time=  11.1s
[CV 1/1] END ..................n_estimators=90;, score=-0.009 total time=  13.8s
[CV 1/1] END .................n_estimators=100;, score=-0.009 total time=  16.7s
{'n_estimators': 100, 'learning_rate': 0.06000000000000001, 'max_depth': 81, 'subsample': 0.5}
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV 1/1] END ...............learning_rate=0.01;, score=-3.717 total time=   2.9s
[CV 1/1] END learning_rate=0.020000000000000004;, score=-0.507 total time=   4.4s
[CV 1/1] END learning_rate=0.0300000000000

Here we now have something interesting. We can see that for `n_estimators` that the score keeps decreasing. This is hinting that we haven't reached the bottom of the validation curve yet, we are still in the underfitting/overfitting region (but here we are actually underfitting). This is also reinforced by the fact that the optimal `n_estimators` is actually the last value we tried. So this is telling us that we should try larger values of `n_estimators`. 

Note that a similar story can be said for the other parameters. But it is complicated by the fact that the optimal `learning_rate` and `subsample` is not actually the last value. This implies that we should focus the search near these points but also try larger values.

In [ ]:
print(base_parameters)

{'n_estimators': 100, 'learning_rate': 0.09000000000000001, 'max_depth': 11, 'subsample': 0.8}


In [ ]:
parameter_grid = {'n_estimators': np.arange(100, 200 + 1, 10),
                  'learning_rate': np.arange(0.07, 0.2, 0.01),
                  'max_depth': np.arange(1, 100 + 1, 10),
                  'subsample': np.arange(0.5, 1, 0.1)}

parameter_grid

{'learning_rate': array([0.07, 0.08, 0.09, 0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17,
        0.18, 0.19]),
 'max_depth': array([ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91]),
 'n_estimators': array([100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]),
 'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])}

In [ ]:
print(base_parameters)
for parameter, grid in parameter_grid.items():
    base_model = XGBRegressor(objective='reg:squarederror', random_state=0, **base_parameters)
    
    parameter_sub_grid = {parameter: grid}
    search = GridSearchCV(base_model, parameter_sub_grid, 
                          cv=[(sampled_train_indices, sampled_valid_indices)], 
                          scoring="neg_mean_squared_error", verbose=3, refit=False)
    
    search.fit(sampled_x_train, sampled_y_train)
    
    base_parameters.update(search.best_params_)
    print(base_parameters)

{'n_estimators': 100, 'learning_rate': 0.09000000000000001, 'max_depth': 11, 'subsample': 0.8}
Fitting 1 folds for each of 11 candidates, totalling 11 fits
[CV 1/1] END .................n_estimators=100;, score=-0.010 total time=  11.3s
[CV 1/1] END .................n_estimators=110;, score=-0.010 total time=  12.4s
[CV 1/1] END .................n_estimators=120;, score=-0.010 total time=  13.6s
[CV 1/1] END .................n_estimators=130;, score=-0.010 total time=  14.7s
[CV 1/1] END .................n_estimators=140;, score=-0.010 total time=  15.9s
[CV 1/1] END .................n_estimators=150;, score=-0.010 total time=  17.1s
[CV 1/1] END .................n_estimators=160;, score=-0.010 total time=  18.3s
[CV 1/1] END .................n_estimators=170;, score=-0.010 total time=  19.5s
[CV 1/1] END .................n_estimators=180;, score=-0.010 total time=  20.7s
[CV 1/1] END .................n_estimators=190;, score=-0.010 total time=  22.0s
[CV 1/1] END .................n_es

I would suggest that you re-do the co-ordinate descent again. But we will just leave it as it is.

Now we train on the best parameters. Notice how long this takes

In [ ]:
base_parameters

{'learning_rate': 0.12999999999999998,
 'max_depth': 11,
 'n_estimators': 200,
 'subsample': 0.7999999999999999}

In [ ]:
%%time

final_model = XGBRegressor(objective='reg:squarederror', random_state=0, **base_parameters)
final_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = final_model.predict(perth_test.drop('log10_price', axis=1))

print(np.mean((y - y_pred) ** 2))

0.0075060996446797215
CPU times: user 2min 25s, sys: 206 ms, total: 2min 25s
Wall time: 2min 25s


Again just to prove the point

In [ ]:
%%time
# Change subsample
new_parameters = {'n_estimators': 200, 'learning_rate': 0.12999999999999998, 
                  'max_depth': 11, 'subsample': 0.5}
new_model = XGBRegressor(objective='reg:squarederror', random_state=0, **new_parameters)
new_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = new_model.predict(perth_test.drop('log10_price', axis=1))

print(np.mean((y - y_pred) ** 2))

0.007741369468912832
CPU times: user 2min 55s, sys: 220 ms, total: 2min 55s
Wall time: 2min 54s


In [ ]:
%%time
# Change Max Depth
new_parameters = {'n_estimators': 200, 'learning_rate': 0.12999999999999998, 
                  'max_depth': 21, 'subsample': 0.7999999999999999}
new_model = XGBRegressor(objective='reg:squarederror', random_state=0, **new_parameters)
new_model.fit(perth_train.drop('log10_price', axis=1), perth_train['log10_price'])

y = perth_test['log10_price']
y_pred = new_model.predict(perth_test.drop('log10_price', axis=1))

print(np.mean((y - y_pred) ** 2))

0.0078446913934133
CPU times: user 4min 39s, sys: 275 ms, total: 4min 39s
Wall time: 4min 38s
